In [3]:
from gensim import models

ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

print(ko_model.wv["민법"])
print("신의성실의" in ko_model.wv.key_to_index)

[ 1.31596327e-01  2.22926497e-01  3.86130475e-02  7.84323066e-02
 -5.16258404e-02  7.97712430e-03  1.99839935e-01 -5.34022510e-01
  4.55676764e-01 -1.09818868e-01  2.69600868e-01  1.33270577e-01
 -7.08474442e-02  3.92643005e-01 -8.30028057e-02  1.95343614e-01
  1.23976536e-01  7.39005357e-02 -5.55521846e-01  4.08919267e-02
 -4.10625562e-02  4.59080249e-01  1.87489107e-01  2.35088855e-01
  2.62020618e-01  1.62249303e+00  4.06613946e-02  1.77449897e-01
  7.33493194e-02  1.39033973e-01  8.16323459e-02 -4.35715675e-01
 -3.06389809e-01  6.05222285e-01 -1.34828299e-01 -1.07189439e-01
 -4.52251770e-02 -3.82874668e-01  2.79095650e-01 -4.98226881e-02
 -3.65950465e-01  3.71522725e-01 -1.66791379e-01  5.06538570e-01
  2.26466656e-01  1.24222279e-01 -3.29030573e-01  5.58425188e-02
  6.66163191e-02  1.33904546e-01 -3.03513199e-01 -3.30507636e-01
  1.03688389e-01 -2.50025034e-01  1.09766692e-01 -3.00087482e-01
 -1.08001627e-01 -4.46593836e-02 -3.73714268e-01 -4.91346195e-02
 -1.81760564e-01  7.92532

In [130]:
import requests

def getWikiData(search):
    url = "https://ko.wikipedia.org/w/api.php?action=parse&parse&page="+ search +"&prop=wikitext&formatversion=2&format=json"
    headers = {'Content-Type': 'application/json'}

    response = requests.get(url, headers=headers)

    return response.json(), url



# print(text)

# for key, value in text['parse'].items():
#     print(f'{key}: {value}\n\n')
    
# print(text['parse']['wikitext'])

# from icu_tokenizer import Tokenizer
def contents(text):
    text, url = getWikiData(text)
    from kiwipiepy import Kiwi

    kiwi = Kiwi()
    kiwi.prepare()


    text_wikitext = text['parse']['wikitext']
    result = kiwi.tokenize(text_wikitext)
    return result
# print(result)

# from sklearn.metrics.pairwise import cosine_similarity

# for token in result:
#     if token in ko_model.wv.key_to_index:
#         cosine_sim = cosine_similarity([ko_model.wv[text['parse']['title']]], [ko_model.wv[token]])
#         if cosine_sim > 0.5:
#             print(f'{token}: {cosine_sim}')
#     # else: 
#     #     print(token)

# from gensim import models

# ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

# print(ko_model.wv["민법"])
# print("신의성실의" in ko_model.wv.key_to_index)


In [160]:
import requests

def getWikiData(search):
    url = "https://ko.wikipedia.org/w/api.php?action=parse&parse&page="+ search +"&prop=wikitext&formatversion=2&format=json"
    headers = {'Content-Type': 'application/json'}

    response = requests.get(url, headers=headers)

    return response.json(), url

def contents(text):
    text, url = getWikiData(text)
    from kiwipiepy import Kiwi

    kiwi = Kiwi()
    kiwi.prepare()


    text_wikitext = text['parse']['wikitext']
    result = kiwi.tokenize(text_wikitext)
    return result, url, text
def word_cosine_similarity(result,text):
    
    cosine_list = []
    made_words = []
    temp_word = ""
    bef_tag = ""
    bef_loc = 0
    form_num = 0
    cnt = 0
    cosine = []
    cosine_sum = 0
    from sklearn.metrics.pairwise import cosine_similarity
    for form, tag, start, length in result:

      # 먼저 명사일 때
      if tag in ['NNP', 'NNG']:

        # 명사 여러 개가 띄어쓰기 없이 나왔을 때 한 단어로 취급, 후에 바꿔도 됨(모든 명사 구분하는 걸로)
        if bef_tag == 'NN' and bef_loc == start:
          temp_word += form

        # 띄어쓰기 후에 들어오는 명사
        elif bef_tag == 'NN' and bef_loc != start:
          made_words.append(temp_word)
          temp_word = form
          form_num = 0

        # 관형격 조사 다음에 들어오는 명사
        elif bef_tag == 'JKG' or bef_tag == 'XSN':
          cosine_list.append(form)
          temp_word += form

        # 처음 들어오는 명사
        elif temp_word == "":
          temp_word = form

        # 단어 위치, 단어 개수 초기화
        bef_loc = start + length
        form_num += 1
        bef_tag = 'NN'
        cnt += 1

      # 관형격 조사일 때 ex)신의성실의 원칙
      elif tag == 'JKG' and bef_tag == "NN":
        cosine_list.append(form)
        temp_word += form + " "
        bef_tag = 'JKG'
        form_num += 1

      # ~적
      elif tag == 'XSN' and bef_tag == "NN":
        temp_word += form + " "
        bef_tag = 'XSN'
        form_num += 1


      # 다른 태그 들어왔을때 단어 리스트에 넣기 및 초기화
      # 더 고려할 것 : 몇 개의 형태소로 이루어진 것만 넣을까, 1글자거나 긴 단어는 뺄까, 1 <= form_num <= 4 and ~의 경우 빼기
      else:
        # if temp_word not in made_words:
        del length
        if bef_tag == 'NN' and len(temp_word) > 1:
          made_words.append(temp_word)
          for token in cosine_list:
            if token in ko_model.wv.key_to_index:
                cosine_sim = cosine_similarity([ko_model.wv[text['parse']['title']]], [ko_model.wv[token]])
                if cosine_sum < cosine_sim:
                    cosine_sum = cosine_sim
                    cosine.append([temp_word, cosine_sum])
          # print(temp_word)
        cosine_list = []
        cosine_sum = 0
        temp_word = ""
        bef_tag = ""
        bef_loc = 0
        form_num = 0
    return cosine, made_words
def getWord(text):
    content, url, text = contents(text)
    cosine, made_words = word_cosine_similarity(content, text)
    made_words_set = set(made_words)
    final_words_list = list(made_words_set)
    result = {}

    for token in final_words_list:
        count = 0
        if token in ko_model.wv.key_to_index:
            cosine_sim = cosine_similarity([ko_model.wv[text['parse']['title']]], [ko_model.wv[token]])
            if cosine_sim > 0.2:
                # for form, tag, start, len in result:
                #     if token==form:
                #         count+=1
                # print(f'{token}: {cosine_sim}')
                result[token] = [cosine_sim, url]

    for token in cosine:
        if token[1] > 0.1:
            count = text_wikitext.count(token[0])
            # print(f'{token[0]} : {token[1]}')
            result[token[0]] = [token[1], url]

    return result

In [161]:
result = getWord("민법")
result

{'무과실책임': [array([[0.30464822]], dtype=float32),
  'https://ko.wikipedia.org/w/api.php?action=parse&parse&page=민법&prop=wikitext&formatversion=2&format=json'],
 '계약관련': [array([[0.20538461]], dtype=float32),
  'https://ko.wikipedia.org/w/api.php?action=parse&parse&page=민법&prop=wikitext&formatversion=2&format=json'],
 '질권': [array([[0.39972463]], dtype=float32),
  'https://ko.wikipedia.org/w/api.php?action=parse&parse&page=민법&prop=wikitext&formatversion=2&format=json'],
 '일본': [array([[0.32041776]], dtype=float32),
  'https://ko.wikipedia.org/w/api.php?action=parse&parse&page=민법&prop=wikitext&formatversion=2&format=json'],
 '사적': [array([[0.20261697]], dtype=float32),
  'https://ko.wikipedia.org/w/api.php?action=parse&parse&page=민법&prop=wikitext&formatversion=2&format=json'],
 '법': [array([[0.35164395]], dtype=float32),
  'https://ko.wikipedia.org/w/api.php?action=parse&parse&page=민법&prop=wikitext&formatversion=2&format=json'],
 '중국': [array([[0.27832693]], dtype=float32),
  'https://ko.